# Práctica Métodos de Simulación - Parte 1

## Elena Rivas, Teresa Grau, Ignacio Casso

**Enunciado. Simulación de sucesos discretos.**

Un taller de fabricación se dedica a procesar tres tipos de piezas, para ello el
taller consta de cuatro células de procesamiento.

En el interior de cada célula se dispone de una máquina de procesado, excepto
en la célula 3 formada por dos máquinas con las mismas características, y de un
almacén (de capacidad ilimitada).

La secuencia de fabricación de cada una de las piezas así como los tiempos de
procesado (expresados en minutos y distribuidos según una triangular) en cada
célula se muestran en la siguiente tabla:


**TODO:** Escribir tabla usando sintáxis de hml:

| A | B | C |
| --- | --- | --- |
| a | b | c |

Los tiempos entre llegadas de las piezas al taller tiene carácter aleatorio. En el
fichero llegadas.txt se proporciona una muestra de tiempos entre llegadas de
piezas. Contrástese si la distribución de dichos tiempos es normal (truncada),
weibull o exponencial y estímense los parámetros de la distribución
correspondiente.

El fichero piezas.text incluye un histórico de piezas que han llegado al taller, que
nos permitirá identificar la proporción de piezas que hay de cada uno de los tres
tipos.Los tiempos de transporte de cada pieza entre las diferentes células es de 2
minutos.

A. Suponiendo que el taller trabaja de forma ininterrumpida (hay tres turnos de
trabajadores), simular el comportamiento del sistema durante 2 meses para
estimar el tiempo mínimo, medio y máximo que tardan en fabricarse los tres
tipos de piezas, el número medio de piezas esperando en cada una de las 4
células y la proporción de tiempo que están ociosas las máquinas de
procesado de las células.

B. Calcular las medidas anteriores si el tiempo de transporte de las piezas entre
las distintas células se reduce la mitad.

En caso de que fuese posible disponer de una máquina de procesado adicional,
¿en qué célula sería más beneficioso ponerla?

## Distribución de los tiempos de llegada

Tenemos que comprobar si los tiempos de llegada dados en el fichero llegadas.txt siguen una distribución normal truncada, weibull o exponencial, y estimar los parámetros correspondientes. Para ello podemos usar los contrastes de hipótesis vistos en ...

## Simulación del Sistema

A continuación simulamos el sistema...

Asumimos FIFO para cada celula de procesamiento...

Usamos los tiempos de llegada y las piezas de los ficheros (aunque tambien podriamos generarlos aleatoriamente, conociendo su distribucion)...
**TODO:** Usar un generador aleatorio? Implementarlo nosotros?

Usamos numpy.random.triangular para generar tiempos de procesado aleatorio...
**TODO:** Implementarlo nosotros?

En primer lugar escribimos los generadores de tiempos y piezas...

In [1]:
import numpy as np
import random
import queue
import math

In [2]:
class System():
    
    def __init__(self, transportTime, num_machines):
        self.eventsQueue = queue.PriorityQueue() # TODO: if two events occur at the exact same time,
                                                              # the priority queue will crash
        self.arrivals = open("E1.llegadas.txt","r")
        self.pieces = open("E1.piezas.txt","r")
        # open files
        
        # fabrication sequence for each type of piece
        self.steps = [[0,1,2,3],
                      [0,1,3,1,2],
                      [1,0,2]]
        
        # processing times for each type of piece at each step of its phabrication sequence
        self.processingTimes =[[(6,9,10), (5,8,10), (15,20,25), (8,12,16)],
                           [(1,13,15), (4,6,8), (15,18,21), (6,9,12), (27,30,39)], # El 30 es un 3 en el enunciado, errata
                           [(7,9,11), (7,10,13), (18,23,28)]]
        
        
        # cell queues
        self.queue = [queue.Queue(),queue.Queue(),queue.Queue(),queue.Queue()]
        
        # number of machines in each cell
        self.num_machines = num_machines
        
        # number of idle machines in each clell
        self.idle_machines = num_machines[:] # copy
        
        # transportation time between cells
        self.transportTime = transportTime
        
        # stats counters
        self.minTime = [math.inf, math.inf, math.inf] # min processing type of each type of piece
        self.maxTime = [0,0,0] # max processing type of each type of piece
        self.sumTimes = [0,0,0] # Sum of the processing times of all pieces for each type of piece
        self.numPieces = [0,0,0] # Total number of pieces processed for each type of piece
        
        self.queueSizeOverTime = [0,0,0,0] # like the area under the integral of the size of each queue over time, but discrete
        self.lastUpdate = [0,0,0,0] # Last time the previous counter was updated
        
        self.idleTime = [0,0,0,0] # total time the machine(s) in a cell have been idle
        self.lastUpdateMachine = [0,0,0,0] # Last time the previous counter was updated
        
    # destructor
        # close files

    def simul_main(self,simulationTime):
    
        self.time = 0
        
        self.simulationTime = simulationTime
        
        if simulationTime > 64500:
            print("File is not big enough, generators needed")
            return False
    
        self.nextPiece()
    
        while not self.eventsQueue.empty():
            self.time, _, nextEvent = self.eventsQueue.get()
            nextEvent()
            
    def simul_main_debug(self, simulationTime):
            
        self.time = 0
    
        self.simulationTime = simulationTime
    
        self.nextPiece()
            
    def debug_step(self):
        
        if not self.eventsQueue.empty():
            self.time, eventDescription, nextEvent = self.eventsQueue.get()
            print(self.time)
            print(eventDescription)
            nextEvent()
        
    def nextPiece(self):
        nextArrival = self.timeTillNextArrival()
        nextArrivalTime = self.time + nextArrival
        if nextArrivalTime < self.simulationTime: # TODO: use 86400 (2 months) when we use true distribution instead of files
            nextPieceType = self.nextPieceType()
            nextPiece = (nextArrivalTime, nextPieceType, 0)
            #eventsQueue.put(self.advancePiece(nextPiece), nextArrivalTime)
            #eventsQueue.put(self.nextPiece(), nextArrivalTime)
            # The prority queue as it is breaks if two events happen at the same time
            eventDescription = "piece " + str(nextPiece) + " arrives"
            event = lambda : self.arrivePiece(nextPiece)
            self.eventsQueue.put((nextArrivalTime, eventDescription, event))
    
    def arrivePiece(self,piece):
        self.nextPiece()
        self.advancePiece(piece)
        
    def nextPieceType(self):
        piece = self.pieces.readline()
        return int(piece)-1
        # Types of pieces are shifted so that the first one is 0
        # TODO: use true distribution?
        
    def timeTillNextArrival(self):
        arrival = self.arrivals.readline()
        return float(arrival)
        # TODO: use true distribution?
    
    def advancePiece(self,piece):
        
        _, pieceType, step = piece

        if self.steps[pieceType][step:] == []:
            self.finishPiece(piece)
        else:
            next_cell = self.steps[pieceType][step]
            if step == 0:
                self.enter(next_cell,piece)
            else:
                eventTime = self.time + self.transportTime
                eventDescription = "piece " + str(piece) + " enters cell " + str(next_cell)
                event = lambda : self.enter(next_cell,piece)
                self.eventsQueue.put((eventTime, eventDescription, event))
    
    def finishPiece(self,piece):
        
        arrivalTime, pieceType, _ = piece
        
        totalTime = self.time - arrivalTime
        
        if totalTime < self.minTime[pieceType]: # use dictionaries, since index needs shifting
            self.minTime[pieceType] = totalTime
       
        if totalTime > self.maxTime[pieceType]:
            self.maxTime[pieceType] = totalTime
        
        self.sumTimes[pieceType] += totalTime
        
        self.numPieces[pieceType] += 1
        
    
    def enter(self,cell,piece):
        
        if self.idle_machines[cell] > 0:
            self.process(cell,piece)
        else:
            self.queueSizeOverTime[cell] += self.queue[cell].qsize()*(self.time-self.lastUpdate[cell])
            self.lastUpdate[cell] = self.time
            self.queue[cell].put(piece)

    def process(self,cell,piece):
        
        self.idleTime[cell] += (self.time - self.lastUpdateMachine[cell])*self.idle_machines[cell]
        self.lastUpdateMachine[cell] = self.time
        
        _, pieceType, step = piece
        
        low, mode, upp = self.processingTimes[pieceType][step]
        
        processing_time = np.random.triangular(low,mode,upp)
        
        self.idle_machines[cell] -= 1
        
        eventDescription = "piece " + str(piece) + " done in cell " + str(cell)
        self.eventsQueue.put((self.time+processing_time, eventDescription, lambda : self.done(cell,piece)))
        
    def done(self,cell,piece):
        
        if self.queue[cell].empty():
            
            self.lastUpdateMachine[cell] = self.time
            self.idle_machines[cell] += 1
        
        else:
            
            self.queueSizeOverTime[cell] += self.queue[cell].qsize()*(self.time-self.lastUpdate[cell])
            self.lastUpdate[cell] = self.time
            nextPiece = self.queue[cell].get()
            self.process(cell,nextPiece)
        
        arrivalTime, pieceType, step = piece
        piece = arrivalTime, pieceType, step + 1
        self.advancePiece(piece)
    
    # TODO: pieces finish two seconds later
    
    # processing time (piece)

In [3]:
system = System(2,[1,1,2,1])
system.simul_main_debug(50)

In [4]:
for _ in range(100):
    system.debug_step()

20.3694
piece (20.3694, 2, 0) arrives
29.021197357846816
piece (20.3694, 2, 0) done in cell 1
31.021197357846816
piece (20.3694, 2, 1) enters cell 0
31.3829
piece (31.3829, 0, 0) arrives
31.6520544
piece (31.6520544, 1, 0) arrives
39.8116384
piece (39.8116384, 0, 0) arrives
41.55426361839897
piece (20.3694, 2, 1) done in cell 0
43.55426361839897
piece (20.3694, 2, 2) enters cell 2
49.59208650549597
piece (31.3829, 0, 0) done in cell 0
49.7992314
piece (49.7992314, 1, 0) arrives
51.59208650549597
piece (31.3829, 0, 1) enters cell 1
57.05128051520387
piece (31.6520544, 1, 0) done in cell 0
58.96044995809295
piece (31.3829, 0, 1) done in cell 1
59.05128051520387
piece (31.6520544, 1, 1) enters cell 1
60.96044995809295
piece (31.3829, 0, 2) enters cell 2
64.01608106431878
piece (31.6520544, 1, 1) done in cell 1
65.22159437108373
piece (39.8116384, 0, 0) done in cell 0
66.01608106431878
piece (31.6520544, 1, 2) enters cell 3
67.22159437108373
piece (39.8116384, 0, 1) enters cell 1
69.672662

In [5]:
system.time

143.57012776695984

### Apartado A: Tiempo de transporte = 2 min.

In [6]:
system = System(2,[1,1,2,1])
system.simul_main(60000)

In [ ]:
system.showStats()

### Apartado B: Tiempo de transporte = 1 min.

In [ ]:
system = System(1,[1,1,2,1])
system.simulate()
system.showStats()

## Máquina de procesado adicional

Habrá que definir beneficio. Consideramos solo el tiempo?

In [ ]:
system = System(2,[2,1,2,1])
system.simulate()
system.showStats()

In [ ]:
system = System(2,[1,2,2,1])
system.simulate()
system.showStats()

In [ ]:
system = System(2,[1,1,3,1])
system.simulate()
system.showStats()

In [ ]:
system = System(2,[1,1,2,2])
system.simulate()
system.showStats()